<img width="8%" alt="LinkedIn.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LinkedIn.png" style="border-radius: 15%">

# LinkedIn - Send posts stats to a Google Sheets spreadsheet
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LinkedIn+-+Send+posts+stats+to+a+Google+Sheets+spreadsheet:+Error+short+description">Bug report</a>

**Tags:** #linkedin #profile #post #stats #naas_drivers #automation #content #googlesheets

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-11-30 (Created: 2022-03-17)

**Description:** This notebook automates the process of sending LinkedIn posts to a Google Sheet for easy tracking and analysis.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin, gsheet
import naas
import pandas as pd

### Setup variables
**Pre-requisite**

- [Get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)
- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

**Mandatory**
- `li_at`: Cookie used to authenticate Members and API clients
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation
- `linkedin_url`: This variable represents the LinkedIn profile URL.
- `spreadsheet_url`: Google Sheets spreadsheet URL
- `sheet_name`: Google Sheets sheet name

**Optional**
- `cron`: This variable represents the CRON syntax used to run the scheduler. More information here: https://crontab.guru/#0_12,18_*_*_1-5
- `limit`: number of posts to be retrieved.

In [ ]:
# Mandatory
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_COOKIE_LI_AT"
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_COOKIE_JSESSIONID"
linkedin_url = "https://www.linkedin.com/in/xxxxx/"  # EXAMPLE "https://www.linkedin.com/in/XXXXXX/"
spreadsheet_url = "https://docs.google.com/spreadsheets/d/XXXXXXXXXXXXXXXXXXXX"
sheet_name = "LK_POSTS_FEED"

# Optional
cron = "0 12,18 * * 1-5" #At minute 0 past hour 12 and 18 on every day-of-week from Monday through Friday.
limit = 10

## Model

### Get data from Google Sheet

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
df_gsheet

### Get new posts and update last posts stats

In [ ]:
def get_new_posts(
    df_gsheet,
    li_at,
    JSESSIONID,
    linkedin_url,
    key="POST_URL",
    limit=10,
    sleep=False
):
    # Get updated posts
    df = linkedin.connect(li_at, JSESSIONID).profile.get_posts_feed(
        linkedin_url,
        limit=limit,
        sleep=sleep
    )
    df = pd.concat([df, df_gsheet]).drop_duplicates(key, keep="first")
    return df

df_new = get_new_posts(
    df_gsheet,
    li_at,
    JSESSIONID,
    linkedin_url,
    limit=limit
)
print("Posts:", len(df_new))
df_new

## Output

### Send to Google Sheet

In [ ]:
gsheet.connect(spreadsheet_url).send(df_new, sheet_name=sheet_name, append=False)

### Add scheduler

In [ ]:
naas.scheduler.add(cron="0 8 * * *")

# -> To delete your scheduler, please uncomment the line below and execute this cell
# naas.scheduler.delete()